%pip install -r requirments.txt --force

In [2]:
from flask_socketio import SocketIO, emit, join_room, leave_room
from flask import *
from flask_login import *

In [11]:
app = Flask(__name__)
app.secret_key = "mkqw2o0@#mk12!mk3"
socketio = SocketIO(app)
lm = LoginManager()
lm.init_app(app)

@lm.user_loader
def user_loader(userId):
    userInfo = User.get_user_info(userId)
    return User(userInfo)

@lm.unauthorized_handler
def unauthorized():
    return redirect('/')

#region CLASS

class User(UserMixin):
    def __init__(self, info):
        self.info = info
    
    #region getter

    def get_id(self):
        return self.info['userId']

    #endregion
    
    @staticmethod
    def get_user_info(userId):
        return {'userId': userId}

#endregion

#region route

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/chatroom', methods = ['post'])
def chatroom():
    params = request.form
    userId = params['userId']
    roomId = params['roomId']
    
    session['userId'] = userId
    session['roomId'] = roomId
    
    return render_template(
        'chat.html',
        userId = userId,
        roomId = roomId
    )

@app.route('/login', methods = ['post'])
def login():
    params = request.get_json()
    userId = params['userId']
    
    userInfo = User.get_user_info(userId)
    login_user(User(userInfo))
    return jsonify({'result': 1})

@app.route('/logout')
def logout():
    logout_user()
    return redirect('/')

#endregion

#region socket

@socketio.on('joined', namespace = '/chatroom')
def chat_joined(d):
    roomId = session.get('roomId')
    print('joined', roomId)
    join_room(roomId)
    socketio.emit('join', d)

@socketio.on('msg', namespace = '/chatroom')
def socket_msg(d):
    roomId = session.get('roomId')
    print('msg', d)
    socketio.emit('msg', d, namespace = '/chatroom', room = roomId)

#endregion

if __name__ == '__main__':
    socketio.run(app = app, host = 'localhost', port = 5000, allow_unsafe_werkzeug = True)

Werkzeug appears to be used in a production deployment. Consider switching to a production web server instead.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Aug/2024 09:08:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2024 09:08:42] "POST /login HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2024 09:08:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2024 09:08:44] "GET /logout HTTP/1.1" 302 -
127.0.0.1 - - [06/Aug/2024 09:08:44] "GET / HTTP/1.1" 200 -
